1. 데이터 적재와 전처리

In [6]:
#mnist 데이터 불러오기
import torchvision
import torch
from torchvision import transforms
image_path = '/content/gdrive/MyDrive/data' #바꿔쓰시오
transform = transforms.Compose([transforms.ToTensor()])#Compose는 여러 전처리를 연결하는 함수 #PyTorch 텐서로 바꿔줌

mnist_dataset = torchvision.datasets.MNIST(root=image_path,
train=True,
transform=transform,
download=True)

In [8]:
#원소하나 살펴보기
torch.set_printoptions(edgeitems=1000, linewidth=1000)

next(iter(mnist_dataset)) #원소하나 보기

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.000

In [19]:
#데이터 분할
from torch.utils.data import Subset

mnist_valid_dataset = Subset(mnist_dataset, torch.arange(10000)) #10,000
mnist_train_dataset = Subset(mnist_dataset, torch.arange(10000, len(mnist_dataset))) #10,000-60,000

mnist_test_dataset = torchvision.datasets.MNIST(root=image_path, train=False, transform=transform, download=False)


In [20]:
#데이터 로더
from torch.utils.data import DataLoader

batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset, batch_size, shuffle=False)


2. torch.nn 모듈로 CNN 구현

In [21]:
#CNN
import torch.nn as nn

model = nn.Sequential()
model.add_module('conv1', nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2))
#합성곱 연산은 기본적으로 출력 크기를 줄이는 효과가 있음. 출력 크기를 유지하거나 경계 정보를 보존하기 위해 패딩(padding)을 사용하여 입력의 가장자리에 0을 덧붙임.
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
model.add_module('conv2', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2))
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))

x = torch.ones((64, 1, 28, 28)) #하나의 배치에 64개의 이미지, 1개의 채널, 28x28인 이미지
model(x).shape #64개의 이미지가 64개의 채널을 가진 7by7 행렬로 출력됨


torch.Size([64, 64, 7, 7])

In [22]:
#합성곱층의 출력 살펴보기
model.add_module('flatten', nn.Flatten())

x = torch.ones((64, 1, 28, 28))
model(x).shape #3136 = 64 * 7 * 7 #하나의 이미지가 3136개의 원소를 가지는 벡터형태로 됨.

torch.Size([64, 3136])

In [23]:
#완전연결층과 규제층
model.add_module('fc1', nn.Linear(3136, 1024)) #풀리커넥티드 층
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p=0.5)) #드롭아웃 규제
model.add_module('fc2', nn.Linear(1024, 10))

In [25]:
#모델을 GPU에 올릴 수 있으면 올리고, 아니면 CPU에서 실행
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
#비용함수 정의와 옵티마이저
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#훈련함수 정의와 모델 훈련
def train(model, num_epochs, train_dl, valid_dl):
    loss_hist_train = [0] * num_epochs
    accuracy_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    accuracy_hist_valid = [0] * num_epochs
    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item()*y_batch.size(0)
            is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
            accuracy_hist_train[epoch] += is_correct.sum().cpu()
        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()
        with torch.no_grad():
           for x_batch, y_batch in valid_dl:
               x_batch = x_batch.to(device)
               y_batch = y_batch.to(device)
               pred = model(x_batch)
               loss = loss_fn(pred, y_batch)
               loss_hist_valid[epoch] += loss.item()*y_batch.size(0)
               is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
               accuracy_hist_valid[epoch] += is_correct.sum().cpu()

        loss_hist_valid[epoch] /= len(valid_dl.dataset)
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)

        print(f'에포크 {epoch+1} 정확도: {accuracy_hist_train[epoch]:.4f} 검증 정확도: {accuracy_hist_valid[epoch]:.4f}')
    return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid

torch.manual_seed(1)
num_epochs = 20
hist = train(model, num_epochs, train_dl, valid_dl)

에포크 1 정확도: 0.9488 검증 정확도: 0.9829
에포크 2 정확도: 0.9845 검증 정확도: 0.9876
에포크 3 정확도: 0.9887 검증 정확도: 0.9883
에포크 4 정확도: 0.9912 검증 정확도: 0.9886


In [ ]:
#학습결과 시각화

import matplotlib.pyplot as plt

x_arr = np.arange(len(hist[0])) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist[0], '-o', label='Train loss')
ax.plot(x_arr, hist[1], '--<', label='Validation loss')
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)
ax.legend(fontsize=15)
ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist[2], '-o', label='Train acc.')
ax.plot(x_arr, hist[3], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)

plt.show()


In [ ]:
#숫자 와 라벨 예측
fig = plt.figure(figsize=(12, 4))
for i in range(12):
    ax = fig.add_subplot(2, 6, i+1)
    ax.set_xticks([]); ax.set_yticks([])
    img = mnist_test_dataset[i][0][0, :, :]
    pred = model(img.unsqueeze(0).unsqueeze(1))
    y_pred = torch.argmax(pred)
    ax.imshow(img, cmap='gray_r')
    ax.text(0.9, 0.1, y_pred.item(),
size=15, color='blue',
horizontalalignment='center',
verticalalignment='center',
transform=ax.transAxes)

plt.show()